In [1]:
import pickle 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 

import nltk
from gensim.models import KeyedVectors
nltk.download('punkt')
 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ishan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
data = pd.read_csv("data/capitals.txt", delimiter = ' ')
data.columns = ['city1', 'country1', 'city2' , 'country2']
# print first five elements in the department 
data.head(5)

,city1,country1,city2,country2
0,Athens,Greece,Bangkok,Thailand
1,Athens,Greece,Beijing,China
2,Athens,Greece,Berlin,Germany
3,Athens,Greece,Bern,Switzerland
4,Athens,Greece,Cairo,Egypt


In [3]:
data.shape

(4951, 4)

In [4]:
embeddings = KeyedVectors.load_word2vec_format('data/GoogleNews-vectors-negative300.bin.gz', binary=True)
f = open('data/capitals.txt','r').read()
set_words = set(nltk.word_tokenize(f))
select_words = words = ['King', 'queen', 'oil', 'gas', 'happy', 'sad' , 'city' , 'town', 'village', 'country', 'continient', 'petroleum', 'joyful']

for w in select_words:
    set_words.add(w)



In [5]:
def get_word_embeddings(embeddings):
    word_embeddings = {}
    for word in embeddings.key_to_index:
        if word in set_words:
            word_embeddings[word] = embeddings[word]
    return word_embeddings
    
# testing the function 
word_embeddings = get_word_embeddings(embeddings)
print(len(word_embeddings))


242


In [6]:
pickle.dump( word_embeddings, open( "word_embeddings_subset.p", "wb" ) )

Now we will load the word embeddings as a python dictionary. As stated these have already been obtained through a machine learning algorithm. 

In [7]:
word_embeddings = pickle.load(open("data/word_embeddings_subset.p", "rb"))
len(word_embeddings)

243

Each of the word embedding is a 300 dimensional vector. 

In [8]:
print("dimension: {}".format(word_embeddings['Spain'].shape[0]))

dimension: 300


Now you will write a function that will use the word embeddings topredict relationships among words. 

The input will take as input three words.
The first two are related to each other. 
It will predict a 4th word which is related to third word in a similar manner as the two first two words are related to each other. 
As an example,"Athens is to Greece as Bangkok is to __"?
You wiull write a program that is capable of finding the fourth word. 
We will give you a hint to show how to compute this. 

A smimilar analogy would be the following:
IN chess:
king - man + woman = queen 


The cosine Similarity function is:
cos(theta)A.B/||A||||B|| = sumation of (prodcut) of AB)/ root of sqaure A product squatre  root of B 

A and B represent the word vectors and A or B represent index I of the vector. Note that if A and B are identical, you will get cos(theta) = 1. 

otherwise if they are the opposite, meainng, A = -B, then you would get cos(theta) = -1
if you get cos(theta) = 0 , that means that they are orthogonal (or perpendicular). 
Numbers between 0 and 1 indicates a similarity score. 
Numbers between -1 and 0 indicates a dissimilarity score. 






# Excercise 1 Cosine_Similarity

implement a function that takes in two word vectors and computes the cosine distance. 


In [9]:
def cosine_similarity(A,B):
    '''Input:
            A: a numpy array which corresponds to a word vector
            B: A numpy array which correcsponds to a word vector
        Output:
            cos: numerical number representing the cosine similariy between A and . 
    '''

    ### Start COde Here ### 
    dot = np.dot(A,B)
    norma = np.linalg.norm(A)  
    normb = np.linalg.norm(B)
    cos = dot/(norma*normb)

    return cos

In [10]:
king = word_embeddings['king']
queen = word_embeddings['queen']

cosine_similarity(king, queen)

0.6510956

# Euclidean Distance 
You will now implkement a function that computes the suiimilarity between two vectors using the Euclidean distanc. Euclidean distance is defined as :

d(A,B) = d(B,A) = root of square of (A-B)^2 + (A-B)^2...+(A - B) ^ 2

n i sthe number of elements in the vector 
A and B are the corresponding word vectors. 
The more similar the words, the more likley the Euclidean distance will be close to 0. 

# Euclidean
Implement a function that computesthe Euclidean distance between two vectors. 



In [11]:
def euclidean(A, B):
    '''' INput:
            A: a numpy array which corresponds to a word vector
            B: a numpy aray which correspojnse to a wprd vector
        Output:
        d: numerical number representing the Euclidean distance between A and B. 
    '''

    d = np.sqrt(np.sum((A-B)**2))

    return d

In [12]:
# Test your function
euclidean(king, queen)

2.4796925

# 1.4 FInding the country of each Capital 
Now, you will use the previous  functions to compute similarities between vectors, and use these to find the capital cities of countries. You will write a function that takes in three word, and the embeding dictionary. Your task is to find the capital citiesr. FOr examples, given the following words:

1 Athens 2: Grece 3: Baghdad, 

Your taks is to predict the country4: Iraq.

# Exercise 3 - get_country

### Instructions:
1. TO predict the capital you might want to look at the King - Man + Woman = Queen exampleabove, and implement that scheme into a mathematical functiona, using the word embeddings and a similarity function. 

2. Iterate over the embedings dictionary and compute the cosine similarity score between your vector abd the current word embedding. 
3. You should add a check to make sure that the word you return is not any of the words that you fed in your function. Return the one with the highest score. 


In [33]:
def get_country(city1, country1, city2, embeddings, cosine_similarity = cosine_similarity):
    """

    """
    # storing the city1 country1 and city2 in a set called group
    group = [city1, country1, city2]

    # get embeddings of the city1
    city1_emb = embeddings[city1]

    # get_embedding of country 1
    country1_emb = embeddings[country1]

    #get embedding of city2
    city2_emb = embeddings[city2]

    # get embedding of country2 
    vec = (country1_emb - city1_emb) + city2_emb

    # Intialize the similarity to -1 (it will be replaced by  a smilarities that aree closer to +1)
    similarity = -1

    # intialize country to empty string
    country = ''

    # Loop through all words in the embeddings dictionary
    for word in embeddings.keys():
        #first check if the word is in group
        if word not in group:
             # get the word embedding
             word_emb = embeddings[word]

             # calculate the cosine similarity 
             cur_similarity = cosine_similarity( vec , word_emb)

             if cur_similarity> similarity:
                 similarity = cur_similarity
                 country = (word, similarity)
    return country



In [34]:
get_country('Athens', 'Greece', 'Cairo', word_embeddings)

('Egypt', 0.7626821)

# Model Accuracy 

Checking the model accuracy of the new function on the dataset

Accuracy = correct # of predictions/ Toral # of predictions

In [47]:
def get_accuracy(word_embeddings, data, get_country = get_country):
    ''''''
    #initialize num correct to zero
    num_correct = 0
    # Loop through the rows of the dataframe
    for i , row in data.iterrows():
        city1 = row['city1']

        country1 = row['country1']

        city2 = row['city2']

        country2 = row['country2']

        predcited_country2,_ = get_country(city1,country1, city2, word_embeddings )

        if predcited_country2 == country2:
            # increment the number of correct by 1 
            num_correct +=1

    m = len(data)
    
    #calculate the accuracy by dividing the number correct by m
    accuracy = num_correct/m

    return accuracy

In [48]:
accuracy = get_accuracy(word_embeddings, data)
print(f"Accuracy is {accuracy:.2f}")

Accuracy is 0.92


# Compute PCA

In [103]:
def get_vectors(words, embedding):
    X = np.zeros((1,300))
    for word in words:
        if word in embedding:
            x = embedding[word]
            X = np.row_stack((X,x))
    X = X[1:,:]
    return X



In [121]:
vec = get_vectors(select_words,word_embeddings)

In [122]:
type(vec)

numpy.ndarray

In [123]:
vec_df = pd.DataFrame(vec)

In [124]:
vec_df

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.005249,-0.143555,-0.069336,0.123535,0.131836,-0.088867,-0.071289,-0.216797,-0.197266,0.055664,...,-0.004608,-0.225586,-0.378906,-0.376953,-0.082520,-0.041260,0.167969,-0.046875,0.163086,0.154297
1,-0.139648,0.062256,-0.279297,0.063965,0.044434,-0.154297,-0.184570,-0.498047,0.047363,0.110840,...,-0.195312,-0.345703,0.217773,-0.091797,0.051025,0.061279,0.194336,0.204102,0.235352,-0.051025
2,-0.161133,0.105469,-0.138672,0.010803,0.017090,-0.041504,-0.145508,-0.341797,-0.028564,0.184570,...,-0.111328,-0.218750,-0.203125,0.140625,-0.041992,0.026611,0.120605,0.255859,0.202148,-0.019287
3,-0.000519,0.160156,0.001610,0.025391,0.099121,-0.085938,0.324219,-0.021729,0.134766,0.110352,...,-0.109863,0.107910,-0.070312,-0.127930,-0.050781,0.042725,-0.073242,-0.035400,0.088867,-0.302734
4,0.189453,0.045898,0.066895,-0.044678,0.178711,0.019653,0.347656,-0.024658,0.406250,0.105957,...,-0.145508,0.033447,0.000477,0.146484,-0.146484,0.033936,-0.052490,-0.062012,0.082520,-0.025513
5,-0.010071,0.057373,0.183594,-0.040039,-0.029785,-0.079102,0.071777,0.013306,-0.143555,0.011292,...,0.024292,-0.168945,-0.062988,0.117188,-0.020508,0.030273,-0.247070,-0.122559,0.076172,-0.234375
6,0.123535,0.159180,0.030029,-0.161133,0.015625,0.111816,0.039795,-0.196289,-0.039307,0.067871,...,-0.007935,-0.091797,-0.265625,0.029297,0.089844,-0.049805,-0.202148,-0.079590,0.068848,-0.164062
7,0.147461,0.205078,0.090332,-0.215820,0.027954,0.198242,0.101562,-0.058105,-0.029663,0.025879,...,-0.193359,-0.156250,-0.349609,-0.117676,-0.110352,0.219727,-0.402344,-0.210938,0.200195,-0.023071
8,-0.080078,0.133789,0.143555,0.094727,-0.047363,-0.023560,-0.008545,-0.186523,0.045898,-0.081543,...,-0.145508,0.067383,-0.244141,-0.077148,0.047607,-0.075195,-0.149414,-0.044189,0.097168,0.067383
9,-0.118652,0.199219,-0.222656,0.108887,0.047852,-0.306641,-0.188477,-0.695312,-0.001259,0.041016,...,-0.179688,-0.361328,0.135742,-0.057373,0.148438,0.026489,0.269531,0.099609,0.206055,-0.099121


In [175]:
def compute_pca(X,n_components = 2):
    """
    """
    # Mean Centerd the data
    X_mean = np.mean(X,axis=0)
    centered_data  = X - X_mean

    # Calculate Covariance matrix
    cov_matrix = np.cov(centered_data)

    # print(len(cov_matrix))
    # print(cov_matrix)


    eigen_vals, eigen_vecs = np.linalg.eig(cov_matrix)
    print(eigen_vals, eigen_vecs)

    

In [176]:

compute_pca(vec)

[6.39961955e-02 4.18138971e-02 2.91606831e-02 1.32556131e-18
 1.62352697e-02 4.18018975e-03 6.33390518e-03 1.34189141e-02
 1.16330674e-02 9.22441351e-03 9.65869391e-03] [[-0.18183541  0.06438431  0.91541842  0.30151134 -0.03757951  0.0155201
  -0.08336587 -0.07817445  0.06347233  0.12256273 -0.01122473]
 [ 0.47831743  0.06019435  0.06009728  0.30151134  0.05318592  0.07960778
   0.80869906 -0.05595704 -0.02001388  0.01114272 -0.07907019]
 [ 0.43017575 -0.02193857  0.00241371  0.30151134 -0.10047565 -0.0824667
  -0.32489684 -0.28923868 -0.39762271 -0.28147654  0.52925104]
 [-0.21903427  0.3228188  -0.151697    0.30151134 -0.23812223 -0.02228091
  -0.04715704 -0.03078001 -0.63243291  0.05472904 -0.52014849]
 [-0.25031449  0.45234324 -0.23012019  0.30151134 -0.51915508 -0.03083124
   0.09325384 -0.0711307   0.44974117  0.02710563  0.31880027]
 [-0.12210989 -0.32803787 -0.11621322  0.30151134  0.04850749 -0.41184862
   0.07221961  0.37263122 -0.16980767  0.58587471  0.29144458]
 [-0.179209